In [5]:
import numpy as np
import random
from random import shuffle
import tensorflow as tf

生成数据，预测问题为给一个长度为20的01序列，计算含有多少个数字1

In [6]:
# Generating the training input data
NUM_EXAMPLES = 7000
# train_input 最后是一个有NUM_EXAMPLES个数的list
# 这里先生成2^15=8192个list，{0：020b},第一个0是第1个参数位置，：是格式限定符，后面代表20位二进制B，以0进行填充
TOTAL_NUMBER=2 ** 13
train_input = ['{0:020b}'.format(i) for i in range(TOTAL_NUMBER)]
# 打乱顺序
shuffle(train_input)
# 将字符串转化为包含int元素的list
train_input = [map(int, i) for i in train_input]
# 将得到的list处理成input形式
ti = []
for i in train_input:
    temp_list = []
    for j in i:
        temp_list.append([j])
    ti.append(np.array(temp_list))
train_input = ti

train_output = []
for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count += 1
    # one hot vector
    temp_list = ([0] * 21)
    temp_list[count] = 1
    train_output.append(temp_list)

In [7]:
# 选取前NUM_EXAMPLES个数据作为训练集，其余作为测试集
test_input = train_input[NUM_EXAMPLES:]
test_output = train_output[NUM_EXAMPLES:]
train_input = train_input[:NUM_EXAMPLES]
train_output = train_output[:NUM_EXAMPLES]

建立computation graph

In [8]:
# The dimensions for data are [Batch Size, Sequence Length, Input Dimension]
data = tf.placeholder(tf.float32, [None, 20, 1])  # Number of examples, number of input in one batch, dimension of each input
target = tf.placeholder(tf.float32, [None, 21]) # 所预测标签的维度

# Creat the LSTM cell
# value for the hidden dimension, or the number of units in the LSTM cell
num_hidden = 24

cell = tf.contrib.rnn.LSTMCell(num_hidden, state_is_tuple=True)
# 注意到这里传入的数据是一个batch，将输出存入val，将The final cell state存入state:
val, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)
# 对val做转置，只交换0,1两个维度
val = tf.transpose(val, [1, 0, 2])
# 这样只取最后一行就能得到最后一次输入后的值
last = tf.gather(val, int(val.get_shape()[0]) - 1)

# 将输出的last的value做一个线性函数拟合输出的标签值
# 初值为截断正态分布的变量
weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
# 初值为0.1的偏置项
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))

prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)
cross_entropy = -tf.reduce_sum(target * tf.log(tf.clip_by_value(prediction, 1e-10, 1.0)))
optimizer = tf.train.AdamOptimizer()

# WARNING
minimize = optimizer.minimize(cross_entropy)

mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

init_op = tf.global_variables_initializer()

c:\users\yang\appdata\local\programs\python\python35\lib\site-packages\tensorflow\python\ops\gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


运行100步

In [9]:
sess = tf.Session()
sess.run(init_op)

batch_size = 1000
no_of_batches = int (len(train_input) / batch_size)
epoch = 100
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr + batch_size], train_output[ptr:ptr + batch_size]
        ptr += batch_size
        sess.run(minimize, {data: inp, target: out})
    if i % 10==0:
        print("Epoch ", str(i))

Epoch  0


Epoch  10


Epoch  20


Epoch  30


Epoch  40


Epoch  50


Epoch  60


Epoch  70


Epoch  80


Epoch  90


测试集误差

In [10]:
incorrect = sess.run(error, {data: test_input, target: test_output})
print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))

Epoch 100 error 9.6%


7对应的概率最高，成功预测

In [11]:
print(sess.run(prediction, {
    data: [[[0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [1], [0], [0], [1], [1], [0], [1], [1], [1], [0]]]}))

[[  3.46713477e-06   6.00579551e-06   2.52224004e-06   1.17098070e-05
    5.93144941e-05   1.81479764e-03   1.04040489e-01   6.93688273e-01
    1.89429447e-01   9.84379090e-03   6.06288319e-04   2.67747266e-04
    2.68845288e-05   1.82004305e-06   1.16824339e-07   8.98250655e-05
    7.46244586e-06   3.18154093e-07   7.57343832e-06   6.84433180e-05
    2.36778651e-05]]


In [12]:
sess.close()